In [16]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch_dsl.connections import connections
import pandas as pd

es = Elasticsearch([{"host":"localhost","port":9200}])
connections.create_connection(hosts=['localhost'], timeout=20)

class Video(Document):
    title = Text()
    channel = Text()
    views = Text()
    date = Text()

    class Index:
        name = 'video_index'

Video.init()

df = pd.DataFrame({0: ['Hogwarts Legacy - Official Launch Trailer 4K', 'Hogwarts Legacy Review', 'Hogwarts Legacy - Before You Buy'],
                1: ['Hogwarts Legacy', 'IGN', 'gameranx'],
                2: ['3,2 M de vues', '1,8 M de vues', '1,8 M de vues'],
                3: ['il y a 6 jours', 'il y a 2 jours', 'il y a 1 jour']})
df.columns = ['title', 'channel', 'views', 'date']

objs = [Video(title=row['title'], channel=row['channel'], views=row['views'], date=row['date'])
        for idx, row in df.iterrows()]

results = Video.search().filter("match", title=df['title'].tolist()).execute()
for result in results:
    print(result.title, result.channel, result.views, result.date)

RequestError: RequestError(400, 'illegal_argument_exception', 'mapper [views] of different type, current_type [text], merged_type [integer]')

In [23]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch_dsl.connections import connections
import pandas as pd

es = Elasticsearch([{"host":"localhost","port":9200}])
connections.create_connection(hosts=['localhost'], timeout=20)

df = pd.DataFrame({'title': ['Hogwarts Legacy - Official Launch Trailer 4K', 'Hogwarts Legacy Review', 'Hogwarts Legacy - Before You Buy'],
                'channel': ['Hogwarts Legacy', 'IGN', 'gameranx'],
                'views': ['3,2 M de vues', '1,8 M de vues', '1,8 M de vues'],
                'date': ['il y a 6 jours', 'il y a 2 jours', 'il y a 1 jour']})

def doc_generator(df):
    df_iter= df.iterrows()
    for index, document in df_iter:
        yield{
            "_index":"video",
            "_type": "_doc",
            "_source": document
        }

heplers.bulk(es, doc_generator(df))

BulkIndexError: ('3 document(s) failed to index.', [{'index': {'_index': 'video', '_type': '_doc', '_id': 'QKvSR4YBhrue8HVRUAeL', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'not_x_content_exception', 'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'}}, 'data': title      Hogwarts Legacy - Official Launch Trailer 4K
channel                                 Hogwarts Legacy
views                                     3,2 M de vues
date                                     il y a 6 jours
Name: 0, dtype: object}}, {'index': {'_index': 'video', '_type': '_doc', '_id': 'QavSR4YBhrue8HVRUAeM', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'not_x_content_exception', 'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'}}, 'data': title      Hogwarts Legacy Review
